# Getting started with the Bedrock SDK

In [3]:
import boto3

def generate_conversation(messages,
                          ):

    bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")
    model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages
    )

    return response["output"]["message"]["content"][0]["text"]

messages = [{
    "role": "user",
    "content": [{"text": "hello world"}]
}]

generate_conversation(messages)

'Hi! How can I help you today?'

In [4]:
messages = [{
    "role": "user",
    "content": [{"text": "What flavors are used in Dr. Pepper?"}]
}]

generate_conversation(messages)

"Dr Pepper's exact formula is a trade secret, but it's known to contain 23 different flavors. While the complete list isn't public, some commonly believed ingredients include:\n\n1. Cherry\n2. Cola\n3. Vanilla\n4. Amaretto/Almond\n5. Blackberry\n6. Caramel\n7. Pepper\n8. Root Beer\n9. Prune juice (though the company has denied this)\n10. Licorice\n\nThe actual combination and proportions of flavors remain confidential, and there may be other flavors not commonly known or speculated about. The unique blend of these flavors creates Dr Pepper's distinctive taste that sets it apart from other soft drinks."

In [5]:
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"

    # Send the message.
response = bedrock_client.converse(
    modelId=model_id,
    messages=[{
        "role": "user",
        "content": [{"text": "hello world"}]
    }],
)

response

{'ResponseMetadata': {'RequestId': '67f892c6-6dcc-4296-a61c-d67449183ab9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 14 Nov 2024 19:19:03 GMT',
   'content-type': 'application/json',
   'content-length': '209',
   'connection': 'keep-alive',
   'x-amzn-requestid': '67f892c6-6dcc-4296-a61c-d67449183ab9'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': 'Hi! How can I help you today?'}]}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 9, 'outputTokens': 12, 'totalTokens': 21},
 'metrics': {'latencyMs': 643}}

In addition to `content`, the response contains some other pieces of information:

* `stopReason` - The reason the model stopped generating.  We'll learn more about this later.
* `usage` - information on billing and rate-limit usage. Contains information on:
    * `inputTokens` - The number of input tokens that were used.
    * `outputTokens` - The number of output tokens that were used.

It's important to know that we have access to these pieces of information, but if you only remember one thing, make it this: `content` contains the actual model-generated content

## Putting words in Claude's mouth



In [6]:
messages = [{
    "role": "user",
    "content": [{"text": "Generate a beautiful haiku"}]
},{
    "role":"assistant",
    "content": [{"text": "calming mountain air"}]   
}]

generate_conversation(messages)

'\ncherry blossoms dance and drift\npeace follows the wind'

In [7]:
print("calming mountain air" + generate_conversation(messages))

calming mountain air  
cherry blossoms dance in dew  
Spring calls in a dream


### Few-shot prompting

One of the most useful prompting strategies is called "few-shot prompting" which involves providing a model with a small number of **examples**.  These examples help guide Claude's generated output.  The messages conversation history is an easy way to provide examples to Claude.

For example, suppose we want to use Claude to analyze the sentiment in tweets.  We could start by simply asking Claude to "please analyze the sentiment in this tweet: " and see what sort of output we get:

In [8]:
messages = [{
    "role": "user",
    "content": [{"text": f"Analyze the sentiment in this tweet: Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"}]
}]

generate_conversation(messages)

'This tweet has a positive sentiment. The use of enthusiastic language ("happy dance"), the positive context of enjoying the product, and the cheerful emoji usage (🌶️🥒) all indicate the user had a favorable experience with the spicy pickles. The inclusion of upbeat hashtags like #pickleslove also reinforces the positive tone.'

This is a great response, but it's probably way more information than we need from Claude, especially if we're trying to automate the sentiment analysis of a large number of tweets.  

We might prefer that Claude respond with a standardized output format like a single word (POSITIVE, NEUTRAL, NEGATIVE) or a numeric value (1, 0, -1).  For readability and simplicity, let's get Claude to respond with either "POSITIVE" or "NEGATIVE".  One way of doing this is through few-shot prompting.  We can provide Claude with a conversation history that shows exactly how we want it to respond: 

```py
messages=[
        {"role": "user", "content": "Unpopular opinion: Pickles are disgusting. Don't @ me"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "I think my love for pickles might be getting out of hand. I just bought a pickle-shaped pool float"},
        {"role": "assistant", "content": "POSITIVE"},
        {"role": "user", "content": "Seriously why would anyone ever eat a pickle?  Those things are nasty!"},
        {"role": "assistant", "content": "NEGATIVE"},
        {"role": "user", "content": "Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"},
    ]
```



In [9]:
messages = [
    {"role": "user", "content": [{"text": "Unpopular opinion: Pickles are disgusting. Don't @ me"}]},
    {"role": "assistant", "content": [{"text": "NEGATIVE"}]},
    {"role": "user", "content": [{"text": "I think my love for pickles might be getting out of hand. I just bought a pickle-shaped pool float"}]},
    {"role": "assistant", "content": [{"text": "POSITIVE"}]},
    {"role": "user", "content": [{"text": "Seriously why would anyone ever eat a pickle?  Those things are nasty!"}]},
    {"role": "assistant", "content": [{"text": "NEGATIVE"}]},
    {"role": "user", "content": [{"text": "Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood"}]}
]

generate_conversation(messages)

'POSITIVE'

***

## Exercise

### Your task: build a chatbot

Build a simple multi-turn command-line chatbot script. The messages format lends itself to building chat-based applications.  To build a chat-bot with Claude, it's as simple as:

1. Keep a list to store the conversation history
2. Ask a user for a message using `input()` and add the user input to the messages list
3. Send the message history to Claude
4. Print out Claude's response to the user
5. Add Claude's assistant response to the history
6. Go back to step 2 and repeat! (use a loop and provide a way for users to quit!)


<details>
    <summary>View exercise solution</summary>

    ```py

        conversation_history = []

        while True:
            user_input = input("User: ")
            
            if user_input.lower() == "quit":
                print("Conversation ended.")
                break
            
            conversation_history.append({
                "role": "user", 
                "content": [{"text": user_input}]
            })
        
            assistant_response = generate_conversation(conversation_history)
            print(f"Assistant: {assistant_response}")
            
            conversation_history.append({
                "role": "assistant", 
                "content": [{"text": assistant_response}]
            })
    ```
</details>

***
